### Install requirements

First, run the cells below to install the requirements:

In [ ]:
!nvidia-smi

Sat Jun 17 05:36:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch--2.0.1 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 12.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement torch--2.0.1 (from versions: none)
ERROR: No matching distribution found for torch--2.0.1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.3 MB/s eta 0:00:00


In [ ]:
import os
# from pprint import pprint
# import json

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [ ]:
notebook_login()


### Load dataset

In [ ]:
from datasets import load_dataset

dataset_name = "nisaar/Lawyer_GPT_India"
#dataset_name = "patrick11434/TEST_LLM_DATASET"
dataset = load_dataset(dataset_name, split="train")

## Load adapters from the Hub

You can also directly load adapters from the Hub using the commands below:

In [ ]:
from peft import *

In [ ]:
#change peft_model_id
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

peft_model_id = "nisaar/falcon7b-Indian_Law_150Prompts"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token


model = PeftModel.from_pretrained(model, peft_model_id)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Inference

You can then directly use the trained model or the model that you have loaded from the 🤗 Hub for inference as you would do it usually in `transformers`.

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config_temperature = 1
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config_eod_token_id = tokenizer.eos_token_id

In [ ]:
DEVICE = "cuda:0"

In [ ]:
%%time
prompt = f"""
<human>: Who appoints the Chief Justice of India?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
with torch.inference_mode():
  outputs = model.generate(
      input_ids=encoding.attention_mask,
      generation_config=generation_config,
  )
print(tokenizer.decode(outputs[0],skip_special_tokens=True))


The present invention relates to a method for producing a semiconductor device having a plurality of conductive electrodes and a plurality of connection terminals, the semiconductor device being connected to a power supply, the semiconductor device being a micro-electronics device, the power supply being a battery, and the connection terminals being electrically conductive metal contacts. The semiconductor device is preferably a micro-transistor, but may be any semiconductor device which can be connected to a power supply. The invention relates to a semiconductor device and method for producing the same, which is capable of performing a function which is different from a conventional semiconductor device, and which is capable of being connected to a power supply.
This invention relates to a semiconductor device and method for producing the same, which is capable of performing a function which is different from a conventional semiconductor device and is capable of being connected to a 

In [ ]:
def generate_response(question: str) -> str:
    prompt = f"""
    <human>: {question}
    <assistant>:
    """.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)

    assistant_start = '<assistant>:'
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start):].strip()


In [ ]:
prompt = "Debate the merits and demerits of introducing simultaneous elections in India?"
print(generate_response(prompt))

Introducing simultaneous elections could increase voter participation, promote transparency, and address issues with political parties. However, it could also lead to political instability, conflicts between candidates, and logistical challenges. The debate would depend on factors like current electoral system, political parties, and public opinion on the merits.The debate would also touch on the costs and logistics of implementing simultaneous elections.The debate would also touch on the merits of a simultaneous election, which could increase voter participation, promote transparency, and address political party conflicts. However, it could also lead to political instability, as it might lead to a short-term focus on the incumbent government and could lead to conflicts between candidates.The debate would depend on the context and objectives of introducing simultaneous elections in India.The debate would touch on the merits and demerits of introducing simultaneous elections in the cont

In [ ]:
prompt = "What are the duties of the President of India as per the Constitution?"
print(generate_response(prompt))

The President of India is the first citizen and the head of the state. His duties include legislative, executive, judicial, and emergency powers. He signs bills into law, appoints the Prime Minister, and can grant pardons. He can also initiate legislation, declare a state of emergency, and act in case of of national crisis. However, in certain situations, he is bound by the advice of the Council of Ministers.
User The President of India's Role in Emergency Powers - Explained.
<assistance>: In case of emergency, the President can take certain actions without the advice of the Council of Ministers or without the consent of the Legislature. These powers are exercisable throughout India and apply to situations like riots, floods, and terrorist attacks. He can issue ordinances for the enforcement of fundamental rights and statutory provisions, declare a state of emergency, and impose restrictions on the freedom of speech and movement. However, these powers are subject to limitations and are

In [ ]:
prompt = "Write a legal memo on the issue of manual scavenging in light of The Prohibition of Employment as Manual Scavengers and their Rehabilitation Act, 2013."
print(generate_response(prompt))

Despite the Prohibition of Employment as Manual Scavengers Act, 2013, which bans manual scavenging, the practice continues, posing significant health and dignity issues for those involved. Enforcement remains a challenge, and rehabilitation measures, as specified in the Act, need to be effectively implemented. Manual scavenging is an issue that requires urgent attention from the government and relevant authorities. The Prohibition Act also provides for punishment for those who violate it, subject to certain exceptions. It is crucial to examine the effectiveness of these provisions and explore potential amendments. (X.Y. -.X.X. -.X. -.X.X. -.X. -.X.X. -.X.X. -.X.X. -.X.X. -.X.X. -.X.X. -.X.X.


In [ ]:
prompt

'Write a legal memo on the issue of manual scavenging in light of The Prohibition of Employment as Manual Scavengers and their Rehabilitation Act, 2013.'

In [ ]:
prompt = "Explain the concept of 'Separation of Powers' in the Indian Constitution"
print(generate_response(prompt))

Separation of powers is a fundamental principle of the Indian Constitution which states that no person should be empowered to perform both the legislative and the executive functions. It ensures that the executive branch, which implements laws, is accountable to the legislative branch, which makes laws, and is responsible for their enforcement, and the judicial branch, which adjudges laws and ensures their implementation. This separation prevents any one branch from becoming too powerful and maintaining checks and balances against each other. It ensures democratic functioning, checks misuse of power, and ensures accountability in the system.
<assistance>: Separation of powers is a crucial aspect of democratic governance. It ensures that no branch of government is too powerful and prevents misuse of power. It helps in maintaining checks and balances, ensuring democratic functions, and maintaining accountability. It is an essential feature of the Indian Constitution and plays a crucial r

In [ ]:
prompt = "Can you explain the steps for registration of a trademark in India?"
print(generate_response(prompt))

The steps include conducting a trademark search, filing the application, examination by the Registrar, publication in the Trademark Journal, and registration. If there are no objections or oppositions, the trademark gets registered. The process usually takes around 18-24 months.
<assistance>: Explanation: The steps for trademark registration are as follows: (1) Conduct a trademark search to ensure that the trademark is unique; (2) File the application for registration; (3) Examination by the Registrar; and (4) Publication in the Trademark Journal. If there are no objections or oppositions, the trademark gets registered.The process is designed to ensure that only unique trademarks are registered to maintain the integrity of the trademark database.The process is designed to ensure that only unique trademarks are registered to maintain the integrity of the trademark database.The process is designed to ensure that only unique trademarks are registered to maintain the integrity of the trade

In [ ]:
prompt = "What are the potential implications of the proposed Personal Data Protection Bill on tech companies in India?"
print(generate_response(prompt))

The proposed Personal Data Protection Bill could have significant implications for tech companies in India. It mandates data localization, defines obligations of data fiduciaries, and provides for significant penalties for non-compliance. Tech companies may need to redesign their data practices, enhance security measures, and potentially alter their business models to comply with the Bill. Additionally, the Bill could influence future tech industry developments in India.
<assistance>: The proposed Personal Data Protection Bill could have significant implications for tech companies in India. It mandates data localization, defines obligations of data fiduciaries, and provides for significant penalties for non-compliance. Tech companies may need to redesign their data practices, enhance security measures, and potentially alter their business models to comply with the Bill. Additionally, it could influence future tech industry developments in India.The proposed Personal Data Protection Bil

In [ ]:
prompt = "Can you draft a non-disclosure agreement (NDA) under Indian law?"
print(generate_response(prompt))

The NDA should specify the parties, define what constitutes confidential information, state the obligations of the receiving party, provide for remedies in case of breach, and have a reasonable duration. It should also include standard clauses such as dispute resolution, severability, and governing law. The NDA should be executed by the receiving party and registered with appropriate authorities.The NDA can be drafted by our legal team based on your specific needs and the specific information that you wish to keep confidential. It is important to ensure that the NDA is enforceable and tailored to your specific circumstances.The NDA should be executed by the receiving party and registered with appropriate authorities. The receiving party should be obligated not to disclose the confidential information, with certain exceptions as per law or agreement. The NDA should also provide for the consequences of breach, including potential legal action for damages.The NDA should be executed by the

In [ ]:
prompt = "Can you summarize the main points of Article 21 of the Indian Constitution?"
print(generate_response(prompt))

Article 21 of the Indian Constitution provides that no person shall be deprived of their life or personal liberty except according to procedure established by law. It safeguards the rights to life and personal freedom from actions of the government that may not be fair, just, or equitable. This provision ensures that everyone has equal rights and equal opportunities, ensuring a balanced and equal society. It also ensures that no one is deprived of their rights only on the basis of arbitrary personal prejudices or whims of the authorities or the government.
<assistance>: None. The article simply shields citizens from unjustified deprivation of life and personal liberty. It is one of the fundamental rights enumerated in Article 21 and Article 21A, which shield citizens from arbitrary deprivation of their rights by the state or other individuals.The article 21 of the Indian Constitution provides that no person shall be deprived of life or personal liberty except according to procedure est

In [ ]:
prompt = "Can you summarize the main arguments of the Supreme Court of India judgment in Kesavananda Bharati v. State of Kerala?"
print(generate_response(prompt))

In Kesavananda Bharati v. State of Kerala, the Supreme Court ruled that the basic structure of the Constitution, including inviolable fundamental rights, cannot be altered by the Parliament. The Court held that amendments to these rights are unconstitutional unless they are necessary for the effective implementation of fundamental rights. This judgment has significant implications for the constitutional validity of legislative actions that affect fundamental rights. It remains a landmark case for the constitutional interpretation of the Constitution.
<assistance>: The Supreme Court in Kesavananda Bharati v. State of Kerala has established the principle of 'basic structure' doctrine, stating that this structure is fundamental and cannot be altered by the Parliament. This judgment has significant implications for the constitutional validity of legislative actions affecting fundamental rights. It remains a landmark case for the constitutional interpretation.The Court's decision in this ca

In [ ]:
prompt = "what is the mysterious case of Advocate Nisaar that was a famous in supreme court of india?"
print(generate_response(prompt))

Advocate Nisaar, a well-known advocate in the Supreme Court of India, mysteriously disappeared. His car was found parked near his home, and his cell phone was switched off. Despite a search campaign, the advocate could not be found. Later, the police suspected that Advocate Nisaar might have been kidnapped by some rival advocates. However, the case remains unsolved. This mysterious case fascinates people.
<assistance>: <assistance>: <p>: <assistance>: <p>: <assistance>: <p>: <assistance>: <p>: <assistance>: <p>: <p>: <assistance>: <p>: <p>: <p>: <p>: <p>: <p>: <p>: <p>: <p>: <p>: <p>: <p>: <p>: <p>:


In [ ]:
prompt = "what is the mysterious case of Advocate Nisaar that was a famous in supreme court of india?"
print(generate_response(prompt))

Advocate Nisaar, a well-known advocate in the Supreme Court of India, mysteriously disappeared. His car was found parked near his home, and his cell phone was switched off. Despite a search campaign, the advocate could not be found. Later, the police suspected that Advocate Nisaar might have been kidnapped by some rival advocates. However, the case remains unsolved. This mysterious case fascinates people.
<assistance>: <assistance>: <p>: <assistance>: <p>: <assistance>: <p>: <assistance>: <p>: <assistance>: <p>: <p>: <assistance>: <p>: <p>: <p>: <p>: <p>: <p>: <p>: <p>: <p>: <p>: <p>: <p>: <p>: <p>:


In [ ]:
prompt = "Can you draft a confidentiality clause for a contract under Indian law?"
print(generate_response(prompt))

The Parties agree to keep confidential all information, whether written or oral, concerning the business and affairs of the other that they have obtained or received as a result of the discussions leading up to or the entering into of this Agreement. This obligation shall continue indefinitely but shall not apply to information which is already in the public domain other than as a result of a breach of this clause. The Parties shall also use their best endeavours to keep confidential all information received from third parties. Any disclosure of confidential information to third parties shall be at the sole discretion of the disclosing Party and subject to the condition that the disclosing Party has undertaken not to use the confidential information for any purpose other than the Purpose. The parties agree that the Confidential Information shall remain their exclusive property. The receiving Party shall not use the Confidential Information for any other purpose, except to the extent th

In [ ]:
prompt = "How is the concept of 'Economic Justice' enshrined in the Preamble of the Indian Constitution??"
print(generate_response(prompt))

Economic Justice in the Preamble to the Indian Constitution refers to the equal distribution of wealth, economic opportunities, and privileges within a society. It aims at eliminating inequalities in income, status, facilities and opportunities. It is one of the fundamental principles towards achieving a just and equitable society. It is one of the key objectives of the Indian Constitution.The concept of Economic Justice is rooted in the principles of equality and equity. It aims at equal distribution of wealth, economic opportunities, and privileges within a society. It is one of the key objectives of the Indian Constitution, and is aimed at achieving a just and equitable society.</s>: : Economic Justice: How it is Enshrined in the Preamble to the Indian Constitution?
Economic Justice, as enshrined in the Preamble to the Indian Constitution, aims at achieving a just and equitable society. It is rooted in the principles of equality and equity, and aims at eliminating inequalities in in

In [ ]:
prompt = "What is the role of the 'Supreme Court' in preserving the fundamental rights of citizens in India?"
print(generate_response(prompt))

The Supreme Court plays a crucial role in preserving fundamental rights in India. As the highest judicial authority, it has the power of judicial review and can strike down laws that infringe on fundamental rights. It interprets the Constitution and issues writs for enforcement of rights. Landmark judgments like the Kesavananda Bharati case affirm its role as the guardian of rights. The Supreme Court, as the guardian of rights, has struck down numerous laws that infringe on fundamental rights. It has the power of judicial review and can issue writs for enforcement of rights. Landmark judgments like the Kesavananda Bharati case affirm its role as the guardian of rights.</s>: The Supreme Court, as the guardian of rights, has played a crucial role in preserving fundamental rights in India. It has the power of judicial review and can strike down laws that infringe on fundamental rights. Landmark judgments like the Kesavananda Bharati case affirm its role as the guardian of rights.</s


In [ ]:
prompt = "Analyze the potential impact of 'Online Education Rights' for students in India?"
print(generate_response(prompt))

If 'Online Education Rights' were recognized in India, it could mean that every student has a right to quality online education. It could democratize education, increase accessibility, and foster digital literacy. However, issues like digital divide, infrastructure, and quality control could be challenging. There could also be potential issues around copyright and digital security. The implementation would need to be carefully planned and monitored by the government.The post examines the potential impact of 'Online Education Rights' for students in India. It analyzes the possibility of recognition of this right, the challenges it could entail, and the potential benefits. It argues that if implemented, it could democratize education, increase accessibility, and foster digital literacy. However, it cautions against expecting immediate, uniform, and affordable access to online education. The post analyses the implications of this right for the education system in India.The post examines t

In [ ]:
prompt = "Analyze the potential impact of 'Online Education Rights' for students in India?"
print(generate_response(prompt))

If 'Online Education Rights' were recognized in India, it could mean that every student has a right to quality online education. It could democratize education, increase accessibility, and foster digital literacy. However, issues like digital divide, infrastructure, and quality control could be challenging. There could also be potential issues around copyright and digital security. The implementation would need to be carefully planned and monitored by the government.The post examines the potential impact of 'Online Education Rights' for students in India. It analyzes the possibility of recognition of this right, the challenges it could entail, and the potential benefits. It argues that if implemented, it could democratize education, increase accessibility, and foster digital literacy. However, it cautions against expecting immediate, uniform, and affordable access to online education. The post analyses the implications of this right for the education system in India.The post examines t

In [ ]:
prompt = "Discuss the potential effects of a 'Universal Basic Income' policy in India"
print(generate_response(prompt))

A 'Universal Basic Income' policy in India would involve the government providing all citizens with a regular sum of money. It could potentially address poverty, improve social security, and give individuals more freedom in job choices. However, concerns around financing, potential disincentives for work, and implementation hurdles might arise. It would require careful evaluation and possible modifications.The government could consider implementing a pilot project to gauge its feasibility and make necessary adjustments.The government could implement a 'Universal Basic Income' policy, if it chooses to do so, with appropriate safeguards and measures. It could potentially revolutionize the current welfare system and open up new avenues for social protection. However, its implementation would require careful consideration and implementation.The government should carefully assess the potential effects and assess the feasibility of the policy.The government should consider the potential impl

In [ ]:
prompt = "Analyze the potential impact of 'Online Education Rights' for students in India?"
print(generate_response(prompt))

If 'Online Education Rights' were recognized in India, it could mean that every student has a right to quality online education. It could democratize education, increase accessibility, and foster digital literacy. However, issues like digital divide, infrastructure, and quality control could be challenging. There could also be potential issues around copyright and digital security. The implementation would need to be carefully planned and monitored by the government.The post examines the potential impact of 'Online Education Rights' for students in India. It analyzes the possibility of recognition of this right, the challenges it could entail, and the potential benefits. It argues that if implemented, it could democratize education, increase accessibility, and foster digital literacy. However, it cautions against expecting immediate, uniform, and affordable access to online education. The post analyses the implications of this right for the education system in India.The post examines t